In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import pickle
import time
import timeit
import h5py
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras


from physionet_processing import (fetch_h5data, spectrogram, 
                                  special_parameters, transformed_stats)

from physionet_processing import (zero_filter, extend_ts, 
                                  random_resample, spectrogram, norm_float)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score
#levinthal 
hd_file = "/scratch/thurasx/ecg_project_2/cnn_ecg_keras/physio.h5"
label_file = "/scratch/thurasx/ecg_project_2/cnn_ecg_keras/REFERENCE-v3.csv"
FILENAME = "11"
BINARY = FALSE
# mac 
# hd_file = "/Users/macbookpro/Documents/physio.h5"
# label_file = "/Users/macbookpro/Documents/ecg_project_2/cnn_ecg_keras/REFERENCE-v3.csv"
h5file =  h5py.File(hd_file, 'r')
# Open hdf5 file


# Get a list of dataset names 
dataset_list = list(h5file.keys())

# Load the labels
label_df = pd.read_csv(label_file, header = None, names = ['name', 'label'])
# Filter the labels that are in the small demo set
label_df = label_df[label_df['name'].isin(dataset_list)]


# Encode labels to integer numbers
label_set = list(sorted(label_df.label.unique()))
encoder = LabelEncoder().fit(label_set)
label_set_codings = encoder.transform(label_set)
label_df = label_df.assign(encoded = encoder.transform(label_df.label))
labels = dict(zip(label_df.name, label_df.encoded))







with open(f"/scratch/thurasx/ecg_project_2/cnn_ecg_keras/cnn_ecg_testing/cnn_small_{FILENAME}_testlabel.pcl", "rb") as f:
    test_labels = pickle.load(f)

# test_labels = list(label_df.iloc[id_train,].name) + list(label_df.iloc[id_val,].name) + list(label_df.iloc[id_test,].name)
# print(len(test_labels))
max_length = 18286

# Output dimensions
sequence_length = max_length
spectrogram_nperseg = 64 # Spectrogram window
spectrogram_noverlap = 32 # Spectrogram overlap
n_classes = len(label_df.label.unique())
batch_size = 32

# calculate image dimensions
data = fetch_h5data(h5file, [0], sequence_length)
_, _, Sxx = spectrogram(data, nperseg = spectrogram_nperseg, noverlap = spectrogram_noverlap)
dim = Sxx[0].shape




batch_size= batch_size
dim= dim
nperseg= spectrogram_nperseg
noverlap= spectrogram_noverlap
n_channels= 1
sequence_length= sequence_length
n_classes=n_classes
shuffle= True
data_mean = -9.01
data_std = 9.00
# Initialization
augment = False
batch_size = len(test_labels)
X = np.empty((batch_size, *dim, n_channels), dtype = float)
y = np.empty((batch_size), dtype = int)
for i, ID in enumerate(test_labels):
    # if i == 10:
    #     break
    data = extend_ts(h5file[ID]['ecgdata'][:, 0], sequence_length)
    data = np.reshape(data, (1, len(data)))

    if augment:
    
        # dropout bursts
        data = zero_filter(data, threshold = 2, depth = 10)
    
        # random resampling
        data = random_resample(data)
    
    # Generate spectrogram
    data_spectrogram = spectrogram(data, nperseg = nperseg, noverlap = noverlap)[2]
    
    # Normalize
    data_transformed = norm_float(data_spectrogram, data_mean, data_std)

    X[i,] = np.expand_dims(data_transformed, axis = 3)
    # print(X[i,].shape)
    # Assuming that the dataset names are unique (only 1 per label)
    y[i] = labels[ID]
    
if BINARY:
    n_classes = 2
    for i, elm in enumerate(y):
        if elm == 0 or elm == 1 or elm == 3:
            y[i] = 0
        elif elm == 2:
            y[i] = 1
# X = X.reshape(batch_size,570,-1)
X_test = X
y_test = keras.utils.to_categorical(y, num_classes=n_classes)


In [ ]:
FILENAME = 'full'
model = tf.keras.models.load_model(f'/scratch/thurasx/ecg_project_2/cnn_ecg_keras/cnn_ecg_keras_tflites/keras_ecg_cnn_small_{FILENAME}.tflite')

y_pre = model.predict(X_test)
y_pred = [y_pre[i].argmax()for i in y_pre]
y_true = [y_test[i].argmax()for i in y_test]
labels = ['Other', 'Normal', 'Atrial Fibrillation', 'Noise']
print("F1 score: ", f1_score(y_true, y_pred, average='weighted'), FILENAME)
print("Accuracy: ", accuracy_score(y_true, y_pred), "Name: ", FILENAME)
cm = confusion_matrix(y_true, y_pred, labels)
cmd = ConfusionMatrixDisplay(cm)
cmd.plot()

In [ ]:
# Load the TFLite model and allocate tensors.

interpreter = tf.lite.Interpreter(model_path=f'/scratch/thurasx/ecg_project_2/cnn_ecg_keras/cnn_ecg_keras_tflites/keras_ecg_cnn_small_{FILENAME}.tflite')
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
print(input_shape)
# print(input_shape.reshape(570,33,1))

# inference
ret_results = []
for i,data in enumerate(X_test):  
    # print(data[i].reshape(540,33))
    # data = data.reshape(1,540,33,1)
    
    X = np.empty((1, 570, 33, 1), dtype = float)
    X[0,] = data
    start = timeit.timeit()
    input_data = X.astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    end = timeit.timeit()
    ret_results.append([start, end, y_test[i].argmax(), output_data.argmax()])

# print(ret_results)
# with open(f"/Users/macbookpro/Documents/ecg_project_2/cnn_ecg_keras/cnn_ecg_testing/small_{FILENAME}_results.pcl", "wb") as f:
#     pickle.dump(ret_results, f)


y_pred = [i[3] for i in ret_results]
y_true = [i[2] for i in ret_results]
labels = ['Other', 'Normal', 'Atrial Fibrillation', 'Noise']
print("F1 score: ", f1_score(y_true, y_pred, average='weighted'), FILENAME)
print("Accuracy: ", accuracy_score(y_true, y_pred), "Name: ", FILENAME)
cm = confusion_matrix(y_true, y_pred, labels)
cmd = ConfusionMatrixDisplay(cm)
cmd.plot()